In [12]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score,accuracy_score
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
import time, os, random, sys
import math
import hyperopt.tpe

import warnings

warnings.filterwarnings('ignore')
np.random.seed(1)

In [37]:
def get_leaf_indices(ensemble, x):
    x = x.astype(np.float32)
    trees = ensemble.estimators_
    n_trees = trees.shape[0]
    indices = []

    for i in range(n_trees):
        tree = trees[i][0].tree_
        indices.append(tree.apply(x))

    indices = np.column_stack(indices)
    return indices



def GBDT_lr(File):
    
    #XGBoost

    start = time.clock()
    train_df, test_df = train_test_split(File, train_size = 0.75)
    X_train = train_df.drop(['cardio'], axis=1)
    y_train = train_df['cardio']
    X_test = test_df.drop(['cardio'], axis=1)
    y_test = test_df['cardio']
    
    
    dtrain = xgb.DMatrix(X_train.values,y_train.values)
    dtest = xgb.DMatrix(X_test.values, y_test.values)
    param = {'silent': 1, 'objective':'binary:logistic','max_depth': 7 }
    watchlist = [(dtest, 'eval'), (dtrain, 'train')]
    num_round=30
    bst= xgb.train(param, dtrain,num_round)


    
    y_pred = bst.predict(dtrain, pred_leaf=True)

    num_leaf= np.max(y_pred)
    X_train_leaves_xgb = np.zeros([len(y_pred),len(y_pred[0]) * num_leaf],dtype=np.int64)
    for i in range(0,len(y_pred)):
        temp = np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[i])
        X_train_leaves_xgb[i][temp] += 1

    y_pred= bst.predict(dtest, pred_leaf=True)    
    X_test_leaves_xgb = np.zeros([len(y_pred),len(y_pred[0]) * num_leaf],dtype=np.int64)
    for i in range(0,len(y_pred)):
        temp = np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[i])
        X_test_leaves_xgb[i][temp] += 1
    
    
    #lightGBM
        
        # create dataset for lightgbm
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

    # specify your configurations as a dict
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': {'binary_logloss'},
        'num_leaves': 63,
        'num_trees': 30,
        'learning_rate': 0.01,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0
    }

    # number of leaves,will be used in feature transformation
    num_leaf = 63


    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=100,
                    valid_sets=lgb_train,
                   verbose_eval=False)

    y_pred = gbm.predict(X_train,pred_leaf=True)
    

    X_train_leaves_lgb = np.zeros([len(y_pred),len(y_pred[0]) * num_leaf],dtype=np.int64)
    for i in range(0,len(y_pred)):
        temp = np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[i])
        X_train_leaves_lgb[i][temp] += 1

    
    y_pred = gbm.predict(X_test,pred_leaf=True)

    X_test_leaves_lgb = np.zeros([len(y_pred),len(y_pred[0]) * num_leaf],dtype=np.int64)
    for i in range(0,len(y_pred)):
        temp = np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[i])
        X_test_leaves_lgb[i][temp] += 1

 
    #GBC

    gbclf=GradientBoostingClassifier(n_estimators=30, max_depth=7,verbose=0)
    gbclf.fit(X_train,y_train)
    leaf = get_leaf_indices
    
    
    
    
    
    
    y_pred=leaf(gbclf,X_train.values)
    num_leaf=np.max(y_pred)
    X_train_leaves_gbc = np.zeros([len(y_pred),len(y_pred[0]) * num_leaf],dtype=np.int64)
    for i in range(0,len(y_pred)):
        temp = np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[i])
        X_train_leaves_gbc[i][temp] += 1

    y_pred=leaf(gbclf,X_test.values)  
    X_test_leaves_gbc = np.zeros([len(y_pred),len(y_pred[0]) * num_leaf],dtype=np.int64)
    for i in range(0,len(y_pred)):
        temp = np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[i])
        X_test_leaves_gbc[i][temp] += 1
    
    
    
    

    X_train_leaves =  np.hstack([X_train_leaves_xgb, X_train_leaves_lgb,X_train_leaves_gbc])
    X_test_leaves= np.hstack([X_test_leaves_xgb, X_test_leaves_lgb,X_test_leaves_gbc])
    
    

    
    bestC=0
    auc_best=0
    
    #---------------------------------------------------------------------------------------------
    #regularization applied testing
    c = np.array([1,0.5,0.1,0.05,0.01,0.005,0.001])
    for t in range(0,len(c)):
        lm = LogisticRegression(penalty='l1',C=c[t]) # logestic model construction
        lm.fit(X_train_leaves,y_train)  # fitting the data

        y_pred_est = lm.predict_proba(X_test_leaves)   # Give the probabilty on each label

        auc=roc_auc_score(y_test, y_pred_est[:,1])
        if auc_best < auc:
            auc_best=auc
            bestC=c[t]
            acc=accuracy_score(y_test,y_pred_est[:,1].round())

    #---------------------------------------------------------------------------------------------    
    
    
    print('best C value: %.2f' % bestC)
    print('GBDT+LR auc 1: %.5f' % auc_best)
    print('GBDT+LR accuracy: %.5f' % acc)
    f_time =time.clock()-start
    print('GBDT+LR time taken: %.2f'% f_time)
    
    
    '''
    lr = LogisticRegression(n_jobs=-1)
    X_train_ext = hstack([X_train_leaves, X_train])
    lr.fit(X_train_ext, y_train)
    X_test_ext = hstack([X_test_leaves, X_test])
    y_pred_gbdtlr2 = lr.predict_proba(X_test_ext)[:, 1]
    gbdtlr_auc2 = roc_auc_score(y_test, y_pred_gbdtlr2)
    print('GBDT+LR auc 2: %.5f' % gbdtlr_auc2)
    f_time =time.clock()-start
    print('GBDT+LR time taken: %.2f'% f_time)
    '''
    
    #lightGBM
    start = time.clock()
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    
    
    
    # specify your configurations as a dict
    params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
    
    }

    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=20,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=5,
                    verbose_eval=False)



    y_pred_lgb = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    # eval
    lgb_auc=roc_auc_score(y_test,y_pred_lgb)
    print('lightGBM auc : %.5f' % lgb_auc)
    
    #XGB
    start = time.clock()
    xgbc=XGBClassifier()
    xgbc.fit(X_train,y_train)
    Y_pred_xgb=xgbc.predict_proba(X_test)[:,1]
    xgb_auc= roc_auc_score(y_test,Y_pred_xgb)
    print('XGB auc : %.5f' % xgb_auc)
    

In [17]:
example = pd.read_csv('train.csv', delimiter=';')

In [18]:
example=example.drop(['id'],axis=1)
example.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [19]:
example.loc[(example.height < 100), 'height'] += 100

In [20]:
example['ap_hi'] = example.ap_hi.abs()
example['ap_lo'] = example.ap_lo.abs()

In [21]:
while np.any(example.ap_hi > 800):
    example.loc[example.ap_hi > 800, 'ap_hi'] /= 10

In [22]:
for i in range(3):
    example.loc[example.ap_hi < 22, 'ap_hi'] *= 10

In [23]:
while np.any(example.ap_lo > 600):
    example.loc[example.ap_lo > 600, 'ap_lo'] /= 10

In [24]:
for i in range(3):
    example.loc[example.ap_lo < 11, 'ap_lo'] *= 10


In [25]:
example['bmi'] = example.weight / ((example.height / 100) ** 2)

In [26]:
example.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,bmi
0,18393,2,168,62.0,110.0,80.0,1,1,0,0,1,0,21.967120
1,20228,1,156,85.0,140.0,90.0,3,1,0,0,1,1,34.927679
2,18857,1,165,64.0,130.0,70.0,3,1,0,0,0,1,23.507805
3,17623,2,169,82.0,150.0,100.0,1,1,0,0,1,1,28.710479
4,17474,1,156,56.0,100.0,60.0,1,1,0,0,0,0,23.011177


In [19]:
GBDT_lr(example)

best C value: 0.00
GBDT+LR auc 1: 0.80140
GBDT+LR time taken: 274.53


In [23]:
GBDT_lr(example)

best C value: 0.00
GBDT+LR auc 1: 0.80030
GBDT+LR time taken: 264.23
lightGBM auc : 0.79764
XGB auc : 0.79996


In [27]:
GBDT_lr(example)

best C value: 0.00
GBDT+LR auc 1: 0.79855
GBDT+LR time taken: 327.45
lightGBM auc : 0.79509
XGB auc : 0.79868


In [28]:
GBDT_lr(example)

best C value: 0.00
GBDT+LR auc 1: 0.79981
GBDT+LR time taken: 323.43
lightGBM auc : 0.79752
XGB auc : 0.80043


In [30]:
GBDT_lr(example)

best C value: 0.00
GBDT+LR auc 1: 0.80149
GBDT+LR time taken: 387.61
lightGBM auc : 0.79955
XGB auc : 0.80205


In [32]:
GBDT_lr(example)

best C value: 0.00
GBDT+LR auc 1: 0.80933
GBDT+LR time taken: 334.32
lightGBM auc : 0.80645
XGB auc : 0.80834


In [24]:
example['cardio'].describe()

count    70000.000000
mean         0.499700
std          0.500003
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: cardio, dtype: float64

In [34]:
GBDT_lr(example) #l1

best C value: 0.05
GBDT+LR auc 1: 0.80232
GBDT+LR time taken: 304.33
lightGBM auc : 0.80134
XGB auc : 0.80460


In [35]:
GBDT_lr(example)

best C value: 0.05
GBDT+LR auc 1: 0.80700
GBDT+LR time taken: 306.92
lightGBM auc : 0.80381
XGB auc : 0.80672


In [36]:
GBDT_lr(example)

best C value: 0.05
GBDT+LR auc 1: 0.80575
GBDT+LR time taken: 306.73
lightGBM auc : 0.80355
XGB auc : 0.80621


In [5]:
GBDT_lr(example)

best C value: 0.05
GBDT+LR auc 1: 0.80223
GBDT+LR time taken: 309.47
lightGBM auc : 0.80126
XGB auc : 0.80431


In [6]:
GBDT_lr(example)

best C value: 0.05
GBDT+LR auc 1: 0.80256
GBDT+LR time taken: 306.94
lightGBM auc : 0.80049
XGB auc : 0.80329


In [9]:
GBDT_lr(example)

best C value: 0.05
GBDT+LR auc 1: 0.79832
GBDT+LR time taken: 244.82
lightGBM auc : 0.79824
XGB auc : 0.80052


In [15]:
GBDT_lr(example) #feature added

best C value: 0.05
GBDT+LR auc 1: 0.80256
GBDT+LR accuracy: 0.74057
GBDT+LR time taken: 253.57
lightGBM auc : 0.80049
XGB auc : 0.80329


In [31]:
GBDT_lr(example)

best C value: 0.05
GBDT+LR auc 1: 0.80358
GBDT+LR accuracy: 0.73880
GBDT+LR time taken: 186.32
lightGBM auc : 0.80271
XGB auc : 0.80550


In [32]:
GBDT_lr(example)

best C value: 0.05
GBDT+LR auc 1: 0.80120
GBDT+LR accuracy: 0.73583
GBDT+LR time taken: 183.67
lightGBM auc : 0.79915
XGB auc : 0.80167


In [34]:
GBDT_lr(example)

best C value: 0.05
GBDT+LR auc 1: 0.80141
GBDT+LR accuracy: 0.73560
GBDT+LR time taken: 182.61
GBDT+LR auc 2: 0.78501
GBDT+LR time taken: 210.00
lightGBM auc : 0.80005
XGB auc : 0.80298


In [39]:
GBDT_lr(example)

best C value: 0.05
GBDT+LR auc 1: 0.80137
GBDT+LR accuracy: 0.73297
GBDT+LR time taken: 234.43
lightGBM auc : 0.79889
XGB auc : 0.80189
